<a href="https://colab.research.google.com/github/Sangeetha2998/Ask-emily/blob/main/emil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
from IPython.display import clear_output

# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 0s (1,448 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initial

In [3]:
from IPython.display import clear_output
!ollama pull llama3.1:8b
clear_output()

In [4]:
!pip install -U lightrag[ollama]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.1/159.1 kB 7.7 MB/s eta 0:00:00


In [5]:
!pip install streamlit
!pip install hugchat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 4.6 MB/s eta 0:00:00


In [6]:
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient

import time
from IPython.display import Markdown, display

# Updated QA template for the Ask Emily project
qa_template = r"""<SYS>
You are Ask Emily, an AI-powered Trip Planner, Travel Advisor, GPS, and RV Technician Finder.
You provide personalized trip recommendations, integrate GPS navigation for RVs, and help users find RV repair technicians.
Answer the user's questions based on the context of travel planning and RV maintenance.
</SYS>
:User  {{input_str}}
You:"""

class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input: dict) -> str:
        return self.generator.call({"input_str": str(input)})

    async def acall(self, input: dict) -> str:
        return await self.generator.acall({"input_str": str(input)})

# Initialize the model client for Ask Emily
model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.1:8b"}
}
qa = SimpleQA(**model)

# Example user query relevant to the Ask Emily system
user_query = "Can you recommend a scenic spot near london?"
output = qa.call(user_query)

# Display the output in Markdown format
display(Markdown(f"**Answer:** {output.data}"))

**Answer:** I'd be delighted to suggest some stunning spots near London!

Considering your love for scenic views, I've got three fantastic recommendations:

1. **Box Hill**: Located in Surrey, about an hour's drive from central London. This picturesque hill offers breathtaking views of the surrounding countryside and the North Downs. On a clear day, you can see as far as London's skyline!
2. **Heathrow Countryside Centre**: Situated near Heathrow Airport, this hidden gem is perfect for a relaxing walk or picnic. The centre features beautiful gardens, tranquil lakes, and stunning views of the surrounding countryside.
3. **Cobham Woods**: Nestled in the heart of Surrey, about 45 minutes from central London, Cobham Woods offers an enchanting forest experience. Take a leisurely stroll through the woods, and enjoy the picturesque scenery, including the River Mole.

These spots are all easily accessible by car or public transportation. Would you like me to provide more details on how to get there?

In [12]:
!pip install Flask flask-ngrok

In [28]:
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient
from IPython.display import Markdown, display

# Updated QA template for the Ask Emily project
qa_template = r"""<SYS>
You are Ask Emily, an AI-powered Trip Planner, Travel Advisor, GPS, and RV Technician Finder.
You provide personalized trip recommendations, integrate GPS navigation for RVs, and help users find RV repair technicians.
Answer the user's questions based on the context of travel planning and RV maintenance.
</SYS>
:User {{input_str}}
You:"""

class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input_str: str) -> str:
        try:
            response = self.generator.call({"input_str": input_str})
            return response.data
        except Exception as e:
            print(f"Error: {e}")
            return "Sorry, an error occurred while processing your request."

# Initialize the model client for Ask Emily
model_client = OllamaClient()
model_kwargs = {"model": "llama3.1:8b"}
qa = SimpleQA(model_client=model_client, model_kwargs=model_kwargs)

# Interactive loop for asking questions
print("Welcome to Ask Emily! Type your travel or RV question below.")
print("Type 'exit' to quit.")

while True:
    # Get the user's question
    user_query = input("You: ")

    # Exit the loop if the user types 'exit'
    if user_query.lower() == 'exit':
        print("Thank you for using Ask Emily. Safe travels!")
        break

    # Generate the response
    output = qa.call(user_query)

    # Display the answer
    display(Markdown(f"**Answer:** {output}"))


Welcome to Ask Emily! Type your travel or RV question below.
Type 'exit' to quit.
You: hi


**Answer:** Hello! I'm Ask Emily, your personal trip planner and RV travel companion. How can I assist you today? Are you planning a new adventure in an RV or do you have some questions about RV maintenance or repair? Let's get started!

You: scenic places in london


**Answer:** London! What a fantastic destination!

I'd be delighted to recommend some of London's most scenic spots for you. Here are a few:

1. **Hyde Park**: One of London's largest parks, featuring beautiful gardens, a picturesque lake, and plenty of green space for picnics or strolls.
2. **Regent's Canal**: A tranquil waterway that winds its way through the city, offering lovely views of London's architecture and a peaceful escape from the hustle and bustle.
3. **Primrose Hill**: A charming hill in North London with stunning 360-degree views of the city, especially beautiful during sunset.
4. **Kew Gardens**: A UNESCO World Heritage Site and one of the largest botanical gardens in the world, featuring over 30,000 plant species and a serene atmosphere.
5. **Little Venice**: A picturesque area where the Grand Union and Regent's Canals meet, offering beautiful views and a charming atmosphere.

These are just a few of the many scenic spots London has to offer. What type of experience are you looking for (e.g., outdoor activities, cultural attractions, relaxation)?

You: outdoor


**Answer:** Welcome to Ask Emily! I'm excited to help you plan your next outdoor adventure. What kind of trip are you thinking about? Are you looking for a relaxing camping getaway, an action-packed hiking trip, or perhaps a scenic road trip in your RV?

What's on your mind? Do you have any specific destinations or activities in mind?

You: exit
Thank you for using Ask Emily. Safe travels!


In [7]:
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient

import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

# Define the QA template for the Ask Emily project
qa_template = r"""<SYS>
You are Ask Emily, an AI-powered Trip Planner, Travel Advisor, GPS, and RV Technician Finder.
You provide personalized trip recommendations, integrate GPS navigation for RVs, and help users find RV repair technicians.
Answer the user's questions based on the context of travel planning and RV maintenance.
</SYS>
:User {{input_str}}
You:"""

class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input_str: str) -> str:
        try:
            response = self.generator.call({"input_str": input_str})
            return response.data
        except Exception as e:
            print(f"Error: {e}")
            return "Sorry, an error occurred while processing your request."

# Initialize the model client for Ask Emily with the specified model
model_client = OllamaClient()
model_kwargs = {"model": "llama3.1:8b"}
qa = SimpleQA(model_client=model_client, model_kwargs=model_kwargs)

# Create the widgets for the chat interface
input_box = widgets.Text(placeholder='Type your message here...')
send_button = widgets.Button(description='Send')
output_area = widgets.Output()

# Define the function to handle sending messages
def on_send_button_click(b):
    with output_area:
        user_input = input_box.value
        if user_input.lower() == 'exit':
            display(Markdown("**Ask Emily:** Goodbye!"))
            input_box.value = ''
            return

        # Display the user's message
        display(Markdown(f"**You:** {user_input}"))

        # Get the chatbot's response
        response = qa.call(user_input)

        # Display the chatbot's response
        display(Markdown(f"**Ask Emily:** {response}"))

        # Clear the input box for the next message
        input_box.value = ''

# Link the send button to the handler function
send_button.on_click(on_send_button_click)

# Display the UI components
display(Markdown("### Welcome to Ask Emily! Chat with me about travel, GPS navigation, or RV maintenance. Type 'exit' to end the conversation."))
display(input_box, send_button, output_area)


### Welcome to Ask Emily! Chat with me about travel, GPS navigation, or RV maintenance. Type 'exit' to end the conversation.

Text(value='', placeholder='Type your message here...')

Button(description='Send', style=ButtonStyle())

Output()